In [20]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 1.8 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 9.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 5.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 37.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.2/613.2 kB 13.8 MB/s eta 0:00:0000:01


In [21]:
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

In [5]:
## abrir o datase de treino e test
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [6]:
## pre-processamento dos dados
#descrição estátistica das features núméricas
est = train.describe()

print(train.info())

#verificar valores nulos ou NAN
print(train.isnull().sum())

print(test.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int6

In [8]:
# mapear as colunas
col = pd.Series(list(train.columns))

X_train = train.drop(['PassengerId', 'Survived'], axis = 1)

X_test = test.drop(['PassengerId'], axis = 1)

In [9]:
# criar feature

def criar_features(X):
  subs = {'female':1, 'male':0}
  X['mulher'] = X['Sex'].replace(subs)
  
  X['Fare'] = X['Fare'].fillna(X['Fare'].mean())
  
  X['Age'] = X['Age'].fillna(X['Age'].mean())
  
  X['Embarked'] = X['Embarked'].fillna('S')
  
  subs = {'S':1, 'C':2, 'Q':3}
  X['porto'] = X['Embarked'].replace(subs)
  
  X['crianca'] = 1
  X['crianca'] = np.where(X['Age'] < 12, 1, 0)
  
  return X

X_train = criar_features(X_train)
X_test = criar_features(X_test)

In [10]:
#Selecionar as features
features = ['Pclass', 'Age', 'SibSp',
       'Parch', 'Fare', 'mulher', 'porto', 'crianca']

X_train = X_train[features]
X_test = X_test[features]

y_train = train['Survived']

In [12]:
gp = train.groupby(['Survived']).count()

#%% pivot_table

table = pd.pivot_table(train, index = ['Survived'], columns = ['Pclass'], values = 'PassengerId', aggfunc = 'count')


#%% Padronização das variáveis

scaler = StandardScaler() #media 0 e desvio padrão 1

X_train_sc = scaler.fit_transform(X_train)

X_test_sc = scaler.transform(X_test)

In [13]:
# modelo e validação cruzada

#Logistic Regression
model_lr = LogisticRegression (random_state= 0 )

score = cross_val_score(model_lr, X_train_sc, y_train, cv = 10)

print(np.mean(score))

#%% Naive Bayes para Classificação

from sklearn.naive_bayes import GaussianNB

model_nb = GaussianNB()

score = cross_val_score(model_nb, X_train_sc, y_train, cv = 10)

print(np.mean(score))

0.8058302122347065
0.8036079900124845


In [14]:
#%% KNN para classificação
from sklearn.neighbors import KNeighborsClassifier

model_knn = KNeighborsClassifier(n_neighbors= 5, p = 2)

score = cross_val_score(model_knn, X_train_sc, y_train, cv = 10)

print(np.mean(score))

0.811498127340824


In [15]:
#%% SVM para classificação
from sklearn.svm import SVC

model_svc = SVC(C = 3, kernel = 'rbf', degree = 2, gamma = 0.1)

score = cross_val_score(model_svc, X_train_sc, y_train, cv = 10)

print(np.mean(score))

0.8316479400749064


In [16]:
#%% Decision Tree

from sklearn.tree import DecisionTreeClassifier

model_dt = DecisionTreeClassifier(criterion = 'entropy', max_depth = 3, min_samples_split = 2, min_samples_leaf = 1, random_state = 0)

score = cross_val_score(model_dt, X_train_sc, y_train, cv = 10)

print(np.mean(score))

0.8170536828963796


In [17]:
#%% Random Forest

from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(criterion = 'entropy', n_estimators = 100, max_depth = 5, min_samples_split = 2, min_samples_leaf = 1, random_state = 0)

score = cross_val_score(model_rf, X_train_sc, y_train, cv = 10)

print(np.mean(score))

0.8261048689138576


In [22]:
def objective(trial):
    # Define os espaços de busca para os parâmetros
    criterion = trial.suggest_categorical('criterion', ['entropy', 'gini'])
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    # Cria o modelo com os parâmetros sugeridos
    model_rf = RandomForestClassifier(
        criterion=criterion,
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=0,
        n_jobs=-1
    )

    # Avalia o modelo usando validação cruzada
    score = cross_val_score(model_rf, X_train_sc, y_train, cv=10)
    mean_score = np.mean(score)

    return -mean_score

study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=0))
study.optimize(objective, n_trials=30)

# Obtém os melhores parâmetros encontrados
best_params = study.best_params
print("Melhores parâmetros encontrados:", best_params)

[I 2023-11-04 17:27:48,517] A new study created in memory with name: no-name-e5358d91-9782-4834-bfa4-3e9825fbb355
[I 2023-11-04 17:28:00,372] Trial 0 finished with value: -0.8238701622971286 and parameters: {'criterion': 'gini', 'n_estimators': 643, 'max_depth': 12, 'min_samples_split': 5, 'min_samples_leaf': 7}. Best is trial 0 with value: -0.8238701622971286.
[I 2023-11-04 17:28:18,021] Trial 1 finished with value: -0.8249937578027465 and parameters: {'criterion': 'gini', 'n_estimators': 968, 'max_depth': 9, 'min_samples_split': 9, 'min_samples_leaf': 6}. Best is trial 1 with value: -0.8249937578027465.
[I 2023-11-04 17:28:21,211] Trial 2 finished with value: -0.8227091136079899 and parameters: {'criterion': 'gini', 'n_estimators': 164, 'max_depth': 4, 'min_samples_split': 2, 'min_samples_leaf': 9}. Best is trial 1 with value: -0.8249937578027465.
[I 2023-11-04 17:28:39,130] Trial 3 finished with value: -0.8238576779026218 and parameters: {'criterion': 'gini', 'n_estimators': 981, 'm

Melhores parâmetros encontrados: {'criterion': 'gini', 'n_estimators': 905, 'max_depth': 12, 'min_samples_split': 7, 'min_samples_leaf': 6}


In [23]:
#%% Ensanble model (Voting)
from sklearn.ensemble import VotingClassifier

model_voting = VotingClassifier(estimators = [('LR', model_lr), ('KNN', model_knn), ('SVC', model_svc), ('RF', model_rf)], voting = 'hard')

model_voting.fit(X_train_sc, y_train)

score = cross_val_score(model_voting, X_train_sc, y_train, cv = 10)

print(np.mean(score))

0.8294257178526842


In [25]:
# Modelo Final
model_rf = RandomForestClassifier(
    criterion=best_params['criterion'],
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    random_state=0,
    n_jobs=-1
)
  
model_rf.fit(X_train_sc, y_train)

y_pred = model_rf.predict(X_train_sc)

mc = confusion_matrix(y_train, y_pred) 
print(mc)

score = model_rf.score(X_train_sc, y_train)
print(score)

#%% predição nos dados de teste

y_pred = model_voting.predict(X_test_sc)

submission = pd.DataFrame(test['PassengerId'])

submission['Survived'] = y_pred

submission.to_csv('submission5.csv', index = False)

[[520  29]
 [ 91 251]]
0.8653198653198653
